In [ ]:
import os
import pandas as pd
import ast
import numpy as np

valid_clfs = [
    "clf_MLP",
    "clf_Linear SVM C0.1",
    "clf_LDA",
    "clf_Random Forest",
]

for root_dir in ["/home/thomas/Dokumente/Master/Master_Thesis/results/BS_Laura/channel_space/loso",
                 "/home/thomas/Dokumente/Master/Master_Thesis/results/BS_Laura/parcel_space/loso",
                  "/home/thomas/Dokumente/Master/Master_Thesis/results/HD_Squeezing/channel_space/loso",
                   "/home/thomas/Dokumente/Master/Master_Thesis/results/HD_Squeezing/parcel_space/loso"]:
    
    print(root_dir)

    results = {}  
    cell_numbers = {}  

    # Traverse directory
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if "sel_hrf_10" in dirpath:
            print(dirpath)
            continue
        for file in filenames:
            if file.endswith(".csv") and "parcel_accuracy_results" in file:
                full_path = os.path.join(dirpath, file)
                parts = full_path.split(os.sep)

                try:
                    ft_folder = next(p for p in parts if p.startswith("ft_"))
                    clf_folder = next(p for p in parts if p.startswith("clf_"))

                    raw_feature = ft_folder[3:]
                    try:
                        ft_name = ast.literal_eval(raw_feature)
                        feature = "_".join(str(x) for x in ft_name) if isinstance(ft_name, (tuple, list)) else str(ft_name)
                    except:
                        feature = raw_feature.replace("'", "").replace("[", "").replace("]", "").replace(",", "_")

                    classifier = clf_folder
                    if classifier in valid_clfs:

                        df = pd.read_csv(full_path)
                        if 'Mean all' in df.columns and 'STD all' in df.columns:
                            mean_vals = df['Mean all'].dropna().values
                            std_vals = df['STD all'].dropna().values

                            if len(mean_vals) == 2 and len(std_vals) == 2:
                                m1, m2 = mean_vals * 100
                                s1, s2 = std_vals * 100
                                text = f"{m1:.0f} {s1:.0f}, {m2:.0f} {s2:.0f}"

                                if classifier not in results:
                                    results[classifier] = {}
                                    cell_numbers[classifier] = {}

                                results[classifier][feature] = text
                                cell_numbers[classifier][feature] = [m1, s1, m2, s2]

                                if classifier == 'clf_Random Forest':
                                    print([m1, s1, m2, s2])

                            else:
                                print(f"Skipping file (not 2 values): {full_path}")
                        else:
                            print(f"Missing columns in {full_path}")

                except Exception as e:
                    print(f"Error: {e} at {full_path}")

    df = pd.DataFrame.from_dict(results, orient='index')
    df = df.reindex(columns=['Slope', 'Slope_Max', 'Slope_Mean', 'Slope_Mean_Max'])
    df = df.reindex(["clf_LDA", "clf_Linear SVM C0.1", "clf_Random Forest", "clf_MLP"])

    import statistics

    numeric_df = pd.DataFrame.from_dict(cell_numbers, orient='index')
    numeric_df = numeric_df.reindex(columns=['Slope', 'Slope_Max', 'Slope_Mean', 'Slope_Mean_Max'])
    numeric_df = numeric_df.reindex(["clf_LDA", "clf_Linear SVM C0.1", "clf_Random Forest", "clf_MLP"])

    # Compute row means (mean over columns)
    row_means = {}
    for clf in numeric_df.index:
        vals = [numeric_df.at[clf, col] for col in numeric_df.columns if isinstance(numeric_df.at[clf, col], list)]
        if vals:
            row_mean = np.mean(vals, axis=0)
            row_means[clf] = row_mean

    # Compute column means (mean over classifiers)
    col_means = {}
    for col in numeric_df.columns:
        vals = [numeric_df.at[clf, col] for clf in numeric_df.index if isinstance(numeric_df.at[clf, col], list)]
        if vals:
            col_mean = np.mean(vals, axis=0)
            col_means[col] = col_mean

    df.loc["Mean"] = {
        col: f"{col_means[col][0]:.0f} {col_means[col][1]:.0f}, {col_means[col][2]:.0f} {col_means[col][3]:.0f}"
        for col in df.columns
    }

    df["Mean"] = [
        f"{row_means[clf][0]:.0f} {row_means[clf][1]:.0f}, {row_means[clf][2]:.0f} {row_means[clf][3]:.0f}"
        if clf in row_means else ""
        for clf in df.index
    ]

    # Add bottom-right cell for mean of means
    overall_vals = list(col_means.values())
    if overall_vals:
        overall_mean = np.mean(overall_vals, axis=0)
        df.at["Mean", "Mean"] = f"{overall_mean[0]:.0f} {overall_mean[1]:.0f}, {overall_mean[2]:.0f} {overall_mean[3]:.0f}"


    df.to_csv("summary_table_with_full_means.csv")
    print("Final table saved as 'summary_table_with_full_means.csv'")
    display(df)

    # Generate LaTeX table string
    latex_lines = []

    # Get the header row with underscores replaced
    header = ["Classifier"] + [col.replace("_", " ") for col in df.columns]
    latex_lines.append(" & ".join(header) + r" \\")

    # Format helper
    def format_cell(val):
        if pd.isna(val):
            return ""
        try:
            parts = val.replace(",", "").split()
            if len(parts) == 4:
                m1, s1, m2, s2 = parts
                return f"{m1} ({s1}) / {m2} ({s2})"
            else:
                return val
        except:
            return val

    # Add rows, replacing _ in classifier names
    for idx, row in df.iterrows():
        formatted_row = [idx.replace("_", " ")] + [format_cell(val) for val in row]
        row_str = " & ".join(formatted_row)
        latex_lines.append(row_str + r" \\")

    # Join and print
    latex_table = "\n".join(latex_lines)
    print("\n✅ Copy-paste ready LaTeX table:")
    print(latex_table)

/home/thomas/Dokumente/Master/Master_Thesis/results/BS_Laura/channel_space/loso
[67.02, 8.97, 69.84, 10.9]
[69.22, 8.459999999999999, 70.59, 10.32]
[68.73, 8.89, 68.66, 10.69]
[72.25, 8.25, 73.27, 11.12]
✅ Final table saved as 'summary_table_with_full_means.csv'


,Slope,Slope_Max,Slope_Mean,Slope_Mean_Max,Mean
clf_LDA,"70 15, 76 11","68 9, 69 10","71 10, 74 11","69 9, 70 10","69 11, 72 11"
clf_Linear SVM C0.1,"72 13, 73 12","66 10, 64 11","72 12, 69 10","64 10, 65 11","68 11, 68 11"
clf_Random Forest,"72 8, 73 11","69 9, 69 11","69 8, 71 10","67 9, 70 11","69 9, 71 11"
clf_MLP,"69 8, 68 7","65 13, 66 12","70 13, 67 14","72 8, 70 10","69 10, 68 11"
Mean,"71 11, 73 10","67 10, 67 11","70 11, 70 12","68 9, 69 11","69 10, 70 11"



✅ Copy-paste ready LaTeX table:
Classifier & Slope & Slope Max & Slope Mean & Slope Mean Max & Mean \\
clf LDA & 70 (15) / 76 (11) & 68 (9) / 69 (10) & 71 (10) / 74 (11) & 69 (9) / 70 (10) & 69 (11) / 72 (11) \\
clf Linear SVM C0.1 & 72 (13) / 73 (12) & 66 (10) / 64 (11) & 72 (12) / 69 (10) & 64 (10) / 65 (11) & 68 (11) / 68 (11) \\
clf Random Forest & 72 (8) / 73 (11) & 69 (9) / 69 (11) & 69 (8) / 71 (10) & 67 (9) / 70 (11) & 69 (9) / 71 (11) \\
clf MLP & 69 (8) / 68 (7) & 65 (13) / 66 (12) & 70 (13) / 67 (14) & 72 (8) / 70 (10) & 69 (10) / 68 (11) \\
Mean & 71 (11) / 73 (10) & 67 (10) / 67 (11) & 70 (11) / 70 (12) & 68 (9) / 69 (11) & 69 (10) / 70 (11) \\
/home/thomas/Dokumente/Master/Master_Thesis/results/BS_Laura/parcel_space/loso
[66.05, 9.27, 70.03, 12.22]
[71.00999999999999, 11.08, 69.73, 13.44]
[68.82000000000001, 12.09, 70.88, 13.11]
[67.64, 12.61, 72.31, 12.049999999999999]
✅ Final table saved as 'summary_table_with_full_means.csv'


,Slope,Slope_Max,Slope_Mean,Slope_Mean_Max,Mean
clf_LDA,"71 11, 76 11","71 9, 71 9","71 10, 76 12","70 11, 72 10","71 10, 74 11"
clf_Linear SVM C0.1,"73 12, 79 11","69 10, 72 10","70 8, 74 11","71 10, 72 10","71 10, 74 10"
clf_Random Forest,"68 13, 72 12","69 12, 71 13","71 11, 70 13","66 9, 70 12","68 11, 71 13"
clf_MLP,"65 12, 69 11","62 10, 69 10","66 12, 69 12","70 10, 67 12","65 11, 68 11"
Mean,"69 12, 74 11","67 10, 70 11","69 10, 72 12","69 10, 70 11","69 11, 72 11"



✅ Copy-paste ready LaTeX table:
Classifier & Slope & Slope Max & Slope Mean & Slope Mean Max & Mean \\
clf LDA & 71 (11) / 76 (11) & 71 (9) / 71 (9) & 71 (10) / 76 (12) & 70 (11) / 72 (10) & 71 (10) / 74 (11) \\
clf Linear SVM C0.1 & 73 (12) / 79 (11) & 69 (10) / 72 (10) & 70 (8) / 74 (11) & 71 (10) / 72 (10) & 71 (10) / 74 (10) \\
clf Random Forest & 68 (13) / 72 (12) & 69 (12) / 71 (13) & 71 (11) / 70 (13) & 66 (9) / 70 (12) & 68 (11) / 71 (13) \\
clf MLP & 65 (12) / 69 (11) & 62 (10) / 69 (10) & 66 (12) / 69 (12) & 70 (10) / 67 (12) & 65 (11) / 68 (11) \\
Mean & 69 (12) / 74 (11) & 67 (10) / 70 (11) & 69 (10) / 72 (12) & 69 (10) / 70 (11) & 69 (11) / 72 (11) \\
/home/thomas/Dokumente/Master/Master_Thesis/results/HD_Squeezing/channel_space/loso
[68.67, 11.21, 76.44, 8.4]
[70.33, 10.9, 77.66999999999999, 7.8]
[70.0, 9.229999999999999, 77.0, 8.61]
[71.33, 11.09, 77.89, 7.41]
✅ Final table saved as 'summary_table_with_full_means.csv'


,Slope,Slope_Max,Slope_Mean,Slope_Mean_Max,Mean
clf_LDA,"76 12, 75 11","73 12, 74 11","72 12, 75 12","71 12, 74 12","73 12, 74 12"
clf_Linear SVM C0.1,"76 12, 79 10","74 12, 77 10","76 12, 78 10","73 12, 77 10","75 12, 78 10"
clf_Random Forest,"71 11, 78 7","70 9, 77 9","70 11, 78 8","69 11, 76 8","70 11, 77 8"
clf_MLP,"74 12, 76 11","67 13, 73 10","73 11, 75 9","71 11, 74 10","71 12, 74 10"
Mean,"74 12, 77 10","71 12, 75 10","73 12, 76 10","71 11, 75 10","72 12, 76 10"



✅ Copy-paste ready LaTeX table:
Classifier & Slope & Slope Max & Slope Mean & Slope Mean Max & Mean \\
clf LDA & 76 (12) / 75 (11) & 73 (12) / 74 (11) & 72 (12) / 75 (12) & 71 (12) / 74 (12) & 73 (12) / 74 (12) \\
clf Linear SVM C0.1 & 76 (12) / 79 (10) & 74 (12) / 77 (10) & 76 (12) / 78 (10) & 73 (12) / 77 (10) & 75 (12) / 78 (10) \\
clf Random Forest & 71 (11) / 78 (7) & 70 (9) / 77 (9) & 70 (11) / 78 (8) & 69 (11) / 76 (8) & 70 (11) / 77 (8) \\
clf MLP & 74 (12) / 76 (11) & 67 (13) / 73 (10) & 73 (11) / 75 (9) & 71 (11) / 74 (10) & 71 (12) / 74 (10) \\
Mean & 74 (12) / 77 (10) & 71 (12) / 75 (10) & 73 (12) / 76 (10) & 71 (11) / 75 (10) & 72 (12) / 76 (10) \\
/home/thomas/Dokumente/Master/Master_Thesis/results/HD_Squeezing/parcel_space/loso
[77.56, 9.76, 79.67, 9.86]
[77.89, 9.32, 80.67, 8.38]
[77.66999999999999, 8.83, 78.89, 9.700000000000001]
[72.89, 10.2, 74.56, 9.73]
[77.44, 8.49, 79.56, 8.67]
✅ Final table saved as 'summary_table_with_full_means.csv'


,Slope,Slope_Max,Slope_Mean,Slope_Mean_Max,Mean
clf_LDA,"75 12, 75 12","72 12, 73 12","74 12, 73 13","73 12, 73 12","73 12, 74 12"
clf_Linear SVM C0.1,"76 11, 79 12","75 12, 77 10","75 12, 76 11","75 11, 77 11","75 11, 77 11"
clf_Random Forest,"77 8, 80 9","78 9, 79 10","78 9, 81 8","78 10, 80 10","78 9, 80 9"
clf_MLP,"72 12, 79 10","74 10, 78 8","74 11, 77 9","73 11, 77 9","73 11, 78 9"
Mean,"75 11, 78 11","75 11, 77 10","75 11, 77 10","75 11, 77 11","75 11, 77 10"



✅ Copy-paste ready LaTeX table:
Classifier & Slope & Slope Max & Slope Mean & Slope Mean Max & Mean \\
clf LDA & 75 (12) / 75 (12) & 72 (12) / 73 (12) & 74 (12) / 73 (13) & 73 (12) / 73 (12) & 73 (12) / 74 (12) \\
clf Linear SVM C0.1 & 76 (11) / 79 (12) & 75 (12) / 77 (10) & 75 (12) / 76 (11) & 75 (11) / 77 (11) & 75 (11) / 77 (11) \\
clf Random Forest & 77 (8) / 80 (9) & 78 (9) / 79 (10) & 78 (9) / 81 (8) & 78 (10) / 80 (10) & 78 (9) / 80 (9) \\
clf MLP & 72 (12) / 79 (10) & 74 (10) / 78 (8) & 74 (11) / 77 (9) & 73 (11) / 77 (9) & 73 (11) / 78 (9) \\
Mean & 75 (11) / 78 (11) & 75 (11) / 77 (10) & 75 (11) / 77 (10) & 75 (11) / 77 (11) & 75 (11) / 77 (10) \\


In [2]:
display(results_bsl_chan[["Slope_Mean_Max", "Slope_Mean", "Slope_Mean", "Mean", "Slope"]].iloc[[0, 1, 4, 5]])

NameError: name 'results_bsl_chan' is not defined

In [ ]:
display(results_hd_sq_chan.iloc[[0, 1, 4, 5]])

,Slope_Mean_Max,Slope_Mean,Slope_Max,Mean,Slope,Mean of row
clf_MLP,"0.674 0.079, 0.729 0.111","0.724 0.108, 0.750 0.109","0.720 0.108, 0.710 0.109","0.666 0.118, 0.661 0.123","0.708 0.110, 0.756 0.110",0.710
clf_Linear SVM C1,"0.693 0.129, 0.722 0.140","0.708 0.125, 0.743 0.117","0.687 0.126, 0.724 0.119","0.704 0.126, 0.729 0.102","0.716 0.127, 0.751 0.141",0.718
clf_LDA,"0.687 0.132, 0.691 0.128","0.706 0.137, 0.717 0.129","0.670 0.146, 0.700 0.136","0.692 0.117, 0.703 0.132","0.706 0.133, 0.716 0.129",0.699
clf_Random Forest,"0.708 0.108, 0.763 0.110","0.716 0.099, 0.781 0.100","0.731 0.099, 0.771 0.088","0.673 0.116, 0.707 0.117","0.723 0.109, 0.767 0.103",0.734


In [ ]:
display(results_bsl_parcel[["Slope_Mean_Max", "Slope_Mean", "Slope_Mean", "Mean", "Slope"]].iloc[[0, 1, 4, 5]])

,Slope_Mean_Max,Slope_Mean,Slope_Mean,Mean,Slope
clf_MLP,"0.673 0.102, 0.669 0.113","0.690 0.119, 0.698 0.109","0.690 0.119, 0.698 0.109","0.588 0.091, 0.621 0.107","0.662 0.126, 0.681 0.108"
clf_Linear SVM C1,"0.714 0.112, 0.693 0.112","0.697 0.114, 0.691 0.089","0.697 0.114, 0.691 0.089","0.640 0.105, 0.694 0.115","0.691 0.097, 0.727 0.135"
clf_LDA,"0.699 0.127, 0.701 0.111","0.704 0.124, 0.745 0.119","0.704 0.124, 0.745 0.119","0.672 0.110, 0.693 0.117","0.703 0.107, 0.754 0.119"
clf_Random Forest,"0.684 0.129, 0.708 0.111","0.662 0.118, 0.714 0.115","0.662 0.118, 0.714 0.115","0.650 0.088, 0.652 0.099","0.692 0.122, 0.728 0.125"


In [ ]:
display(results_hd_sq_parcel.iloc[[0, 1, 4, 5]])

,Slope_Mean_Max,Slope_Mean,Slope_Max,Mean,Slope,Mean of row
clf_MLP,"0.730 0.112, 0.766 0.091","0.743 0.107, 0.773 0.095","0.742 0.103, 0.777 0.077","0.689 0.107, 0.717 0.110","0.723 0.121, 0.792 0.096",0.745
clf_Linear SVM C1,"0.739 0.109, 0.741 0.117","0.738 0.107, 0.753 0.112","0.743 0.113, 0.754 0.108","0.722 0.086, 0.770 0.090","0.758 0.104, 0.792 0.110",0.751
clf_LDA,"0.728 0.120, 0.733 0.124","0.738 0.119, 0.734 0.129","0.724 0.125, 0.734 0.123","0.721 0.097, 0.737 0.108","0.749 0.117, 0.753 0.124",0.735
clf_Random Forest,"0.776 0.098, 0.797 0.099","0.779 0.093, 0.807 0.084","0.777 0.088, 0.789 0.097","0.729 0.102, 0.746 0.097","0.774 0.085, 0.796 0.087",0.777
